### Simple Wrappers

In [1]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents.Wrappers import *

path = str(inspect.getfile(CybORG))
path = '../Shared/Scenarios/Scenario2_No_Decoy.yaml'

cyborg = CybORG(path,'sim')

Our demonstrations have mostly been with native CybORG, but we will now demonstrate how AI can be assisted by the use of Wrappers.

Our first Wrapper is EnumActionWrapper, which calculates all the possible actions and returns the action space as the number of such actions. This gives a relatively large number of actions for both Blue and Red team.

In [6]:
env = EnumActionWrapper(cyborg)

results = env.reset(agent='Blue')
action_space = results.action_space
print('Blue action space:',action_space)

print(env.possible_actions)

Blue action space: 41
[<CybORG.Shared.Actions.Action.Sleep object at 0x7f47c2552fa0>, <CybORG.Shared.Actions.AbstractActions.Monitor.Monitor object at 0x7f47c24ae7c0>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c24ae0a0>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c24ae0d0>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c24a8070>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c24fcbb0>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c24fcd00>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c261caf0>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c261a160>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c24eba00>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c24eb5b0>, <CybORG.Shared.Actions.AbstractActions.Analyse.Analyse object at 0x7f47c250ff40>, <CybORG.Shar

We can do a similar thing for the observation space. The FlatFixedWrapper parses the internal state of CybORG and turns it into a list of floats, which can easily be converted into a vector. Unfortunately, this vector is extremely long, with length over 11,000!

In [35]:
env = BlueTableWrapper(cyborg,output_mode='vector')

env.reset(agent='Blue')
for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation.shape
    print(blue_obs)
    print(76*'-')

(42,)
----------------------------------------------------------------------------
(42,)
----------------------------------------------------------------------------
(42,)
----------------------------------------------------------------------------


In [24]:
env = FixedFlatWrapper(CybORG(path, 'sim'))

results = env.reset()
obs = results.observation
print(type(obs))
print(len(obs))

<class 'list'>
11293


The OpenAIGymWrapper converts the output of FlatFixedWrapper to a numpy array as well as conforming to other parts of the OpenAI Gym API. It requires FlatFixedWrapper and EnumActionWrapper in order to function and should always be the outermost of the provided wrappers. You must also specify an agent parameter and explitly specify the environment parameter.

In [19]:
wrappers = FixedFlatWrapper(EnumActionWrapper(cyborg))
env = OpenAIGymWrapper(env=wrappers,agent_name='Blue')

obs = env.reset()
print('Observation:',obs)
print(73*'-')
print('Action_Space:',env.action_space)
print(73*'-')
print('Observation Space:',env.observation_space)

Observation: [ 0.66666667  0.          1.         ... -1.         -1.
 -1.        ]
-------------------------------------------------------------------------
Action_Space: Discrete(41)
-------------------------------------------------------------------------
Observation Space: Box(-1.0, 1.0, (11293,), float32)


### Table Wrappers

The 'Table Wrappers' attempt to use basic logic to infer a human-friendly picture of the state by keeping track of past observations. This allows for a greatly simplified state space and much greater human readibility. However, it mercilessly exploits the current limitations of Scenario 1b and thus would have limited use on real-world cyber problems.

The first wrapper is the TrueTableWrapper, which modifies the get_agent_state method to return the true state in the form of the table.

In [20]:
env = TrueTableWrapper(cyborg)

env.reset()

true_table = env.get_agent_state('True')
print(true_table)

+----------------+-------------+-------------+-------+---------+------------+
|     Subnet     |  IP Address |   Hostname  | Known | Scanned |   Access   |
+----------------+-------------+-------------+-------+---------+------------+
| 10.0.64.112/28 | 10.0.64.119 |   Defender  | False |  False  |    None    |
| 10.0.64.112/28 | 10.0.64.118 | Enterprise0 | False |  False  |    None    |
| 10.0.64.112/28 | 10.0.64.123 | Enterprise1 | False |  False  |    None    |
| 10.0.64.112/28 | 10.0.64.124 | Enterprise2 | False |  False  |    None    |
| 10.0.221.0/28  |  10.0.221.9 |   Op_Host0  | False |  False  |    None    |
| 10.0.221.0/28  |  10.0.221.7 |   Op_Host1  | False |  False  |    None    |
| 10.0.221.0/28  |  10.0.221.6 |   Op_Host2  | False |  False  |    None    |
| 10.0.221.0/28  |  10.0.221.3 |  Op_Server0 | False |  False  |    None    |
| 10.0.76.176/28 | 10.0.76.188 |    User0    |  True |  False  | Privileged |
| 10.0.76.176/28 | 10.0.76.181 |    User1    | False |  False  |

The BlueTableWrapper provides similar functionality for the blue observation.

In [21]:
from CybORG.Agents import B_lineAgent
from CybORG.Shared.Actions import Sleep
cyborg = CybORG(path,'sim', agents={'Red':B_lineAgent})

env = BlueTableWrapper(cyborg)

results = env.reset(agent='Blue')

for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation
    print(blue_obs)

+-----------------+--------------+-------------+----------+-------------+
|      Subnet     |  IP Address  |   Hostname  | Activity | Compromised |
+-----------------+--------------+-------------+----------+-------------+
| 10.0.150.128/28 | 10.0.150.130 |   Defender  |   None   |      No     |
| 10.0.150.128/28 | 10.0.150.138 | Enterprise0 |   None   |      No     |
| 10.0.150.128/28 | 10.0.150.135 | Enterprise1 |   None   |      No     |
| 10.0.150.128/28 | 10.0.150.131 | Enterprise2 |   None   |      No     |
|   10.0.72.0/28  |  10.0.72.6   |   Op_Host0  |   None   |      No     |
|   10.0.72.0/28  |  10.0.72.12  |   Op_Host1  |   None   |      No     |
|   10.0.72.0/28  |  10.0.72.8   |   Op_Host2  |   None   |      No     |
|   10.0.72.0/28  |  10.0.72.10  |  Op_Server0 |   None   |      No     |
| 10.0.237.128/28 | 10.0.237.131 |    User0    |   None   |      No     |
| 10.0.237.128/28 | 10.0.237.140 |    User1    |   None   |      No     |
| 10.0.237.128/28 | 10.0.237.136 |    

The table can also be converted into a vector. This is done by setting the output_mode parameter to 'vector'.

The RedTableWrapper is the Red Team version of the BlueTableWrapper.

In [8]:
env = RedTableWrapper(cyborg)

results = env.reset(agent='Red')
print(results.observation)

for i in range(3):
    results = env.step(agent='Red')
    red_obs = results.observation
    print(red_obs)

+-----------------+--------------+----------+---------+------------+
|      Subnet     |  IP Address  | Hostname | Scanned |   Access   |
+-----------------+--------------+----------+---------+------------+
| 10.0.219.192/28 | 10.0.219.194 |  User0   |  False  | Privileged |
+-----------------+--------------+----------+---------+------------+
+-----------------+--------------+-----------------+---------+------------+
|      Subnet     |  IP Address  |     Hostname    | Scanned |   Access   |
+-----------------+--------------+-----------------+---------+------------+
| 10.0.219.192/28 | 10.0.219.193 | UNKNOWN_HOST: 2 |  False  |    None    |
| 10.0.219.192/28 | 10.0.219.194 |      User0      |  False  | Privileged |
| 10.0.219.192/28 | 10.0.219.196 | UNKNOWN_HOST: 0 |  False  |    None    |
| 10.0.219.192/28 | 10.0.219.198 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.219.192/28 | 10.0.219.199 | UNKNOWN_HOST: 1 |  False  |    None    |
+-----------------+--------------+-------------

### Challenge Wrapper

The challenge wrapper is three wrappers nested together: BlueTableWrapper, EnumActionWrapper and OpenAIGymWrapper.

In [9]:
env = ChallengeWrapper(env=cyborg,agent_name='Red')

obs = env.reset()

for i in range(1):
    obs, reward, done, info = env.step()
    print(obs)
    print(76*'-')
    print(reward)
    print(76*'-')
    print(done)
    print(76*'-')
    pprint(info)

[ 1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
----------------------------------------------------------------------------
0.0
----------------------------------------------------------------------------
False
----------------------------------------------------------------------------
{'action': <CybORG.Shared.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems object at 0x7fbdfc1879a0>,
 'action_name': None,
 'action_space': 888,
 'done': False,
 'error': None,
 'error_msg': None,
 'info': None,
 'next_observation': None,
 'observation': array([ 1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1]),
 'parameter_mask': None,
 'reward': 0.0,
 'selection_masks': None}


In [15]:
if np.array([0,0,0])

SyntaxError: unexpected EOF while parsing (3060779413.py, line 1)